## seq2seq2 Language translation model with Attention


Sequence(Seq2Seq) models uses encoder-decoder architecture.

A few use cases for seq2seq

    Neural machine translation(NMT),
    Image captioning,
    Chatbots
    Abstractive text summarization etc.


As we know when we read the story at starting we dont have to high attention in the story but as we move to the end of the story out attention get high. Same way Memory of the words at the beginning may not be as strong as word near the end. How can we overcome this issue in RNN. We can use another LSTM layer their we will feed our sentences in oposite directions so this way we combine the 2 timestamps. This is also called bidirectional LSTM.

So now question come arises How do we combine ?




<img src="https://raw.githubusercontent.com/teja/Machine_Learning/master/Images/seq_to_seq_lstm_attention_1.PNG" width="540" height="240" align="left"/>

There are many way to cobine the Hstate and Cstate ate each time stamps.
1. concetenate 
2. Multiplication
3. Sum

We can solve attention word attention at end and starting of sentences using bidirectional LSTM but still we have another problem.

It will not work good with long sentences.As what happend, information of whole sentence gets stored at the hidden and cell state at last time stamp like compressing whole information at the end of the sentence.as shown below

<img src="https://raw.githubusercontent.com/teja/Machine_Learning/master/Images/seq_to_seq_lstm_attention_2.PNG" width="540" height="240" align="left"/>

 We can provide whole information like each words and last time stamp H and C embedding to decoder as below. This way decoder will have more knowledge what are information came and whens o decoder can look these things. 

<img src="https://raw.githubusercontent.com/teja/Machine_Learning/master/Images/seq_to_seq_lstm_attention_3.PNG" width="540" height="240" align="left"/>

 As we know we if we pass all information to decoder then decoder may get confused so we will pass information in the form of weights. It measn we can create a attention matrix. Now question come how we can decide the weights to each hidden state?. 

<img src="https://raw.githubusercontent.com/teja/Machine_Learning/master/Images/seq_to_seq_lstm_attention_4.PNG" width="540" height="240" align="left"/>

We will use Global Attention mechanism for seq2seq language translation. We can read more about this 
https://arxiv.org/pdf/1508.04025.pdf

In [72]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(keras.__version__)
import pandas as pd
import numpy as np
import re

2.2.0
2.3.0-tf


In [0]:
## Import data and extract zip file in CWD
import zipfile, urllib, os,io
from urllib.request import Request,urlopen, urlretrieve
url = "https://raw.githubusercontent.com/teja/Data_Files/master/vocabulary/hin-eng.zip"
 
local_filename, headers = urllib.request.urlretrieve(url=url)
zip_file = zipfile.ZipFile(file = local_filename, mode = 'r')
##zip_file.extractall(path = os.getcwd())     #os.getcwd() directs to current working directory
## If using windows we can use below command but we need to download wget
#!wget https://raw.githubusercontent.com/teja/Data_Files/master/vocabulary/hin-eng.zip --quiet

In [0]:
## Read the text file
data = ""
with zip_file.open("hin.txt") as textfile:
  for line in io.TextIOWrapper(textfile,"utf-8"):
    data += line 

In [75]:
type(data),len(data)

(str, 401016)

In [76]:
data[400:500]

'.\tCC-BY 2.0 (France) Attribution: tatoeba.org #631038 (Shishir) & #6179123 (fastrizwaan)\nHello!\tनमस्'

Extract Language source and Target pairs

In [0]:
## We need to delete the extra string from "\tCC-BY" in each line
## import re
textdata = re.findall(r"^(.*)\tCC-BY",data,re.M)

In [0]:
#Split by newline character
#data =  data.split('\n')

In [0]:
#Show some Data
#data[100:105]

In [0]:
#type(data),len(data)

In [77]:
type(textdata),len(textdata)

(list, 2778)

In [78]:
textdata[10:15]

["I'm OK.\tमैं ठीक हूँ।",
 'Awesome!\tबहुत बढ़िया!',
 'Come in.\tअंदर आ जाओ।',
 'Get out!\tबाहर निकल जाओ!',
 'Go away!\tचले जाओ!']

Separate Source and Target Language Pairs

In [0]:
encoder_text = [] #Initialize Source language list
decoder_text = [] #Initialize Target language list
#Iterate over data
for line in textdata:
    try:
        in_txt, out_txt = line.split('\t')
        encoder_text.append(in_txt)
        
        # Add tab '<start>' as 'start sequence in target
        # And '<end>' as End
        decoder_text.append('<start> ' + out_txt + ' <end>')
    except:
        pass #ignore data which goes into error   

In [13]:
encoder_text[100:105]

['I have a car.',
 'I have a dog.',
 'I understand.',
 "I'm a doctor.",
 'It is a book.']

In [14]:
decoder_text[100:105]

['<start> मेरे पास एक गाड़ी है। <end>',
 '<start> मेरे पास एक कुत्ता है। <end>',
 '<start> मैं समझता हूँ। <end>',
 '<start> मैं डॉक्टर हूँ। <end>',
 '<start> यह किताब है। <end>']

Tokenize the Source Language Sentences

In [0]:
encoder_tk = keras.preprocessing.text.Tokenizer()
encoder_tk.fit_on_texts(encoder_text)

In [0]:
#encoder_tk.word_index
#encoder_tk.word_count

In [0]:
## Convert sentences of tokenizer to number
encoder_seq  = encoder_tk.texts_to_sequences(encoder_text)

In [18]:
type(encoder_seq),len(encoder_seq)

(list, 2778)

In [19]:
encoder_seq[100:105]

[[2, 14, 6, 96], [2, 14, 6, 124], [2, 208], [39, 6, 150], [10, 5, 6, 69]]

In [20]:
#Maximum length of sentence
max_encoder_seq_length = max([len(txt) for txt in encoder_seq])
print('Maximum sentence length for Source language: ', max_encoder_seq_length)

#Source language Vocablury
encoder_vocab_size = len(encoder_tk.word_index)
print('Source language vocablury size: ', encoder_vocab_size)

Maximum sentence length for Source language:  22
Source language vocablury size:  2375


Tokenize the Target Language Sentences

In [0]:
#Tokenizer for target language, filters should not <start> and <end>
#remove < and > used in Target language sequences
decoder_tk = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
decoder_tk.fit_on_texts(decoder_text) #Fit it on target sentences
decoder_seq = decoder_tk.texts_to_sequences(decoder_text) #Convert sentences to numbers 

In [22]:
type(decoder_seq),len(decoder_seq),decoder_seq[100:105]

(list,
 2778,
 [[1, 28, 40, 20, 101, 3, 2],
  [1, 28, 40, 20, 202, 3, 2],
  [1, 6, 762, 22, 2],
  [1, 6, 181, 22, 2],
  [1, 25, 78, 3, 2]])

In [23]:
#Maximum length of sentence
max_decoder_seq_length = max([len(txt) for txt in decoder_seq])
print('Maximum sentence length for Target language: ', max_decoder_seq_length)

#Target language Vocablury
decoder_vocab_size = len(decoder_tk.word_index)
print('Target language vocablury size: ', decoder_vocab_size)

Maximum sentence length for Target language:  27
Target language vocablury size:  2973


Compare Encoder and Decoder sentences lenght


In [24]:
#Source Language sentences
print('Length for sentence number 100: ', len(encoder_seq[100]))
print('Length for sentence number 2000: ', len(encoder_seq[2000]))

Length for sentence number 100:  4
Length for sentence number 2000:  5


In [25]:
#Target Language sentences
print('Length for sentence number 100: ', len(decoder_seq[100]))
print('Length for sentence number 2000: ', len(decoder_seq[2000]))

Length for sentence number 100:  7
Length for sentence number 2000:  9


As we can see there are difference in lenght of sentences and we need to use same length sentence for both encoder and decoder so we will use padding 

### Padding in sentences

In [0]:
## Padding in source sentences: will use length =22 for all encoder sentences
encoder_input_data = tf.keras.preprocessing.sequence.pad_sequences(encoder_seq, 
                                                                   maxlen=max_encoder_seq_length, #22
                                                                   padding='pre')

## Padding "pre"  means data closer to the end point

In [0]:
decoder_input_data = tf.keras.preprocessing.sequence.pad_sequences(decoder_seq, 
                                                                   maxlen=max_decoder_seq_length, #27
                                                                   padding='post')
## Padding "post" means data closer to the start point

In [28]:
print('Source data shape: ', encoder_input_data.shape)
print('Target data shape: ', decoder_input_data.shape)

Source data shape:  (2778, 22)
Target data shape:  (2778, 27)


In [29]:
encoder_text[0],type(encoder_input_data)

('Wow!', numpy.ndarray)

In [30]:
encoder_input_data[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1264],
      dtype=int32)

In [31]:
decoder_text[0],type(decoder_input_data)

('<start> वाह! <end>', numpy.ndarray)

In [32]:
decoder_input_data[0]

array([  1, 750,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0], dtype=int32)

In [0]:
## Integer to word converter for decoder output
#decoder_tk.word_index
int_to_word_decoder = dict((i,c) for c,i in decoder_tk.word_index.items())

In [34]:
int_to_word_decoder[15]

'की'

Prepare Decoder Output

In [0]:
#Initialize array
decoder_target_data = np.zeros((decoder_input_data.shape[0], decoder_input_data.shape[1]))

#Shift Target output by one word
for i in range(decoder_input_data.shape[0]):
    for j in range(1,decoder_input_data.shape[1]):
        decoder_target_data[i][j-1] = decoder_input_data[i][j]

In [36]:
decoder_target_data[0]

array([750.,   2.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.])

In [0]:
#Initialize one hot encoding array
decoder_target_one_hot = np.zeros((decoder_input_data.shape[0], #number of sentences
                                   decoder_input_data.shape[1], #Number of words in each sentence
                                   len(decoder_tk.word_index)+1)) #Vocab size + 1

In [0]:
#Build one hot encoded array
for i in range(decoder_target_data.shape[0]):
    for j in range(decoder_target_data.shape[1]):
        decoder_target_one_hot[i][j] = tf.keras.utils.to_categorical(decoder_target_data[i][j],
                                                                     num_classes=len(decoder_tk.word_index)+1)    

In [39]:
decoder_target_one_hot.shape

(2778, 27, 2974)

Trainning Model

In [0]:
## lets take some input parameter
encoder_embedding_size = 50
decoder_embedding_size = 50
rnn_units = 256 ## It measn H(hidden state in LSTM) and C(cell staye in LSTM) both will have 256 numbers
## LSTM Timestamp is 22 becuase my input language sentence lenght is 22 and we are going to learn max 22 lenght sentence
## This could be change if sentence lenght is more or less

Build Encoder

In [0]:
## Input Layer for Encoder
## Decoder has more than 1 input/output so we are not using Sequential
encoder_inputs = tf.keras.layers.Input(shape=(None,)) ##  As we have 22 length sentence so we can use 22 inplace of None but we we want to work with any number than we should use None.
## Add embedding layer
encoder_embedding = tf.keras.layers.Embedding(encoder_vocab_size+1, encoder_embedding_size)
## Get embedding Layer output by feeding encoder input
encoder_embedding_output = encoder_embedding(encoder_inputs)
## Add LSTM Layer to get H and C and output as shown in below image
## return_state=True,## when we set True we get 3 output(H,C and output)
##encoder_out,state_h, state_c = tf.keras.layers.LSTM(rnn_units,return_state=True)(encoder_embedding_output)
## Build a list of H and C to feed in decoder
##encoder_states = [state_h,state_c]

Build Encoder to get all hidden states

In [0]:
#Create LSTM Layer and get All hidden states, last hidden and cell state
encoder_lstm = tf.keras.layers.LSTM(rnn_units,return_sequences=True,return_state=True)
#Get 3 outputs of LSTM Layer
encoder_all_h_states, state_h, state_c = encoder_lstm(encoder_embedding_output)
#Build a list to feed Decoder
encoder_states = [state_h, state_c]

Build Decoder

In [0]:
## Decoder Input-- Target padded sequences
decoder_inputs = tf.keras.layers.Input(shape=(None,))##As we have 27 length sentence so we can use None inplace of 27 but we we want to work with any number than we should use None.
decoder_embedding = tf.keras.layers.Embedding(decoder_vocab_size + 1, decoder_embedding_size)
## Embedding Layer Output
decoder_embedding_output = decoder_embedding(decoder_inputs)
## Decoder RNN-LSTM 
## return_sequences=True means get all H output from each time stamps.
## return_state=True,when we set True we get H output at each time stmaps as well as last H and C
decoder_rnn = tf.keras.layers.LSTM(rnn_units, return_sequences=True, return_state=True)
#Decoder RNN Output, State initialization from Encoder states
#Output will be all hidden sequences, last 'h' state and last 'c' state
decoder_all_h_states,_,_= decoder_rnn(decoder_embedding_output, initial_state=encoder_states)
## Decoder Output to DENSE layer
##decoder_dense =tf.keras.layers.Dense(decoder_vocab_size + 1, activation='softmax')
## Dense Layer Output
##decoder_outputs = decoder_dense(decoder_output)

In [44]:
decoder_inputs,decoder_embedding,decoder_embedding_output,decoder_rnn,decoder_all_h_states

(<tf.Tensor 'input_2:0' shape=(None, None) dtype=float32>,
 <tf.Tensor 'embedding_1/Identity:0' shape=(None, None, 50) dtype=float32>,
 <tf.Tensor 'lstm_1/Identity:0' shape=(None, None, 256) dtype=float32>)

<img src="https://raw.githubusercontent.com/atulpatelDS/Machine_Learning/master/Images/seq_to_seq_lstm_attention_6.PNG" width="540" height="240" align="left"/>

Build Decoder -- Alignment Matrix

In [0]:
#1. Dot Product between Decoder_all_h_states and encoder_all_h_states
#2. Apply softmax to get Alignment matrix

#Dimensions details
#decoder_all_states = batch_size x max_decoder_length x rnn_units
#encoder_all_states = batch_size x max_encoder_length x rnn_units
#score = batch_size x max_decoder_length x max_encoder_length
#alignment matrix = batch_size x max_decoder_length x max_encoder_length

score = tf.keras.layers.dot([decoder_all_h_states, encoder_all_h_states], axes=2)
alignment_matrix = tf.keras.layers.Activation('softmax')(score)

#Try general and concat approaches to alignment matrix

In [46]:
score,alignment_matrix

(<tf.Tensor 'dot/Identity:0' shape=(None, None, None) dtype=float32>,
 <tf.Tensor 'activation/Identity:0' shape=(None, None, None) dtype=float32>)

<img src="https://raw.githubusercontent.com/atulpatelDS/Machine_Learning/master/Images/seq_to_seq_lstm_attention_7.PNG" width="540" height="240" align="left"/>

Context Vector

In [0]:
#Weighted sum of multiplication of Alignment matrix and encoder states
#Dimension of context_vector =  batch_size x max_decoder_length x rnn_units

context_vector = tf.keras.layers.dot([alignment_matrix, encoder_all_h_states], axes=[2,1])

In [48]:
context_vector

<tf.Tensor 'dot_1/Identity:0' shape=(None, None, 256) dtype=float32>

<img src="https://raw.githubusercontent.com/atulpatelDS/Machine_Learning/master/Images/seq_to_seq_lstm_attention_5.PNG" width="540" height="240" align="left"/>

Attention Vector

In [0]:
#Concatenate context vector and decoder_all_h_states
#context_decoder_hidden = batch_size x max_decoder_length x rnn_units
#attention_vector = batch_size x max_decoder_length x 128

context_decoder_hidden = tf.keras.layers.concatenate([context_vector, 
                                                      decoder_all_h_states])

attention_dense_layer = tf.keras.layers.Dense(128, use_bias=False, 
                                              activation='tanh')

attention_vector = attention_dense_layer(context_decoder_hidden)

In [50]:
context_decoder_hidden,attention_dense_layer,attention_vector

(<tf.Tensor 'concatenate/Identity:0' shape=(None, None, 512) dtype=float32>,
 <tf.Tensor 'dense/Identity:0' shape=(None, None, 128) dtype=float32>)

Decoder Output Layer

In [0]:
#Output layer
decoder_dense = tf.keras.layers.Dense(decoder_vocab_size + 1, activation='softmax')

#With attention input will be attention_vector and not decoder_all_h_states
decoder_outputs = decoder_dense(attention_vector)

In [52]:
decoder_dense,decoder_outputs

(<tensorflow.python.keras.layers.core.Dense at 0x7efb765b15c0>,
 <tf.Tensor 'dense_1/Identity:0' shape=(None, None, 2974) dtype=float32>)

Build seq2seq Model using Both Encoder and Decoder

In [0]:
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], #2 Inputs to the model
                              decoder_outputs) #Output of the model

In [54]:
model.input

[<tf.Tensor 'input_1:0' shape=(None, None) dtype=float32>,
 <tf.Tensor 'input_2:0' shape=(None, None) dtype=float32>]

In [55]:
model.output

<tf.Tensor 'dense_1/Identity:0' shape=(None, None, 2974) dtype=float32>

In [0]:
model.compile(optimizer="adam",loss="categorical_crossentropy")

Train The Model

In [57]:
## English-Source Lang sentence length = 22
## Hindi-Target Lang Sentence Length = 27
## Total count of sentences pairs (English to Hindi) : 2778
## Target Language Vocabulary size+1-- One Hot Encoding :  2974
encoder_input_data.shape,decoder_input_data.shape,decoder_target_one_hot.shape ## Target Language Vocabulary size+1:  2974

((2778, 22), (2778, 27), (2778, 27, 2974))

In [58]:
model.fit([encoder_input_data,decoder_input_data],decoder_target_one_hot,
          batch_size=64,
          epochs=1000,
          validation_split=0.2)

Epoch 1/1000
35/35 [==============================] - 2s 66ms/step - loss: 4.2154 - val_loss: 3.3639
Epoch 2/1000
35/35 [==============================] - 1s 42ms/step - loss: 2.1332 - val_loss: 3.3294
Epoch 3/1000
35/35 [==============================] - 1s 41ms/step - loss: 2.0468 - val_loss: 3.1985
Epoch 4/1000
35/35 [==============================] - 1s 41ms/step - loss: 1.8249 - val_loss: 2.9912
Epoch 5/1000
35/35 [==============================] - 1s 41ms/step - loss: 1.6704 - val_loss: 2.9708
Epoch 6/1000
35/35 [==============================] - 1s 41ms/step - loss: 1.5569 - val_loss: 2.6436
Epoch 7/1000
35/35 [==============================] - 1s 40ms/step - loss: 1.4588 - val_loss: 2.5754
Epoch 8/1000
35/35 [==============================] - 1s 40ms/step - loss: 1.4056 - val_loss: 2.5599
Epoch 9/1000
35/35 [==============================] - 1s 41ms/step - loss: 1.3693 - val_loss: 2.5150
Epoch 10/1000
35/35 [==============================] - 2s 44ms/step - loss: 1.3432 - val_lo

In [59]:
## Save the model
model.save("model/seq2seq_language_translation_lstm_attention.hd5")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model/seq2seq_language_translation_lstm_attention.hd5/assets


## Prediction Model

As we see that we use the Hindi(Target) as input for decoder and in real time we will not get that as input for our model so we can not directly use this using model.predict. We need to make separate prediction model.

Build the Encoder Model to predict Encoder States

In [0]:
encoder_model = tf.keras.models.Model(inputs=encoder_inputs, #Padded input sequences
                                      outputs=[encoder_all_h_states] + #Hidden states at all time steps
                                      encoder_states) #Hidden state and Cell state at last time step

**Build the decoder model**
<p/>
<ol><li>Define Input for both 'h' state and 'c' state initialization </li>
    <li><font color="blue">Define Input for all encoder states - Attention Layer </font></li>
<li>Get Decoder RNN outputs along with h and c state</li>
<li><font color="blue">Build Attention Layer</font></li>
<li><font color="blue">Get Decoder Dense layer output using Attention vector</font></li>
    <li><font color="blue">Build Model</font></li></ol>

***Define input for both H and C state***

In [0]:
#Hidden state input
decoder_state_input_h = tf.keras.layers.Input(shape=(rnn_units,))

#Cell state input
decoder_state_input_c = tf.keras.layers.Input(shape=(rnn_units,))

#Putting it together
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [62]:
decoder_state_input_h,decoder_state_input_c ,decoder_states_inputs

(<tf.Tensor 'input_3:0' shape=(None, 256) dtype=float32>,
 <tf.Tensor 'input_4:0' shape=(None, 256) dtype=float32>,
 [<tf.Tensor 'input_3:0' shape=(None, 256) dtype=float32>,
  <tf.Tensor 'input_4:0' shape=(None, 256) dtype=float32>])

**Define Input encoder states - Attention Layer**

In [63]:
encoder_outputs = tf.keras.layers.Input(shape=(max_encoder_seq_length, rnn_units,))
encoder_outputs

<tf.Tensor 'input_5:0' shape=(None, 22, 256) dtype=float32>

***Get Decoder LSTM output along with H and C state***

In [0]:
#Get Embedding layer output
x = decoder_embedding(decoder_inputs) ## decoder_embedding_output
#We will use the layer which we trained earlier
## return_sequences=True means get all H output from each time stamps.
## return_state=True,when we set True we get H output at each time stmaps as well as last H and C
rnn_outputs, state_h, state_c = decoder_rnn(x, initial_state=decoder_states_inputs)
## As we saw earlier we ignore the state_h,state_c in decoder building but here we are using becuase we use use last state of H and C
## We will initilize the second Decoder layer with the previous H and C as so on as per below image
decoder_states = [state_h, state_c]


***Build Attention Layer***

In [0]:
#Alignment score
p_score = tf.keras.layers.dot([rnn_outputs, encoder_outputs], axes=2)

#Perform softmax to get Alignment matrix
p_alignment_matrix = tf.keras.layers.Activation('softmax')(p_score)

#Context Vector
p_context_vector = tf.keras.layers.dot([p_alignment_matrix, encoder_outputs], axes=[2,1])

#Build Attention Vector
# 1. Caoncatenate both context vector and decoder outputs
# 2. Feed it to the Dense layer 
p_context_decoder_hidden = tf.keras.layers.concatenate([p_context_vector, rnn_outputs])
p_attention_vector = attention_dense_layer(p_context_decoder_hidden)

***Get Decoder Dense layer output***

In [0]:
decoder_outputs = decoder_dense(p_attention_vector)

***Build Decoder Model***

In [0]:
#3 Inputs - Word, h/c state and all hidden states from encoder
#3 Outputs - predicted word, h and c state values for next run and alignment matrix for visualization

decoder_model = tf.keras.models.Model([decoder_inputs] +  #Start sequence and then word
                                      decoder_states_inputs + #h and c state value for initialization
                                      [encoder_outputs],  #Encoder all hidden states for Attention layer
                                      [decoder_outputs] + #Model word prediction
                                      decoder_states +   #h and c states for next run
                                      [p_alignment_matrix]) #for Alignment matrix

***Build Prediction Function***

In [0]:
## Prediction fucntion which will run both encoder and decoder Model
def decoder_output_sentence(input_lang_sequence):
  ## Get the encoder state output : sentence embedding (h and c)
  encoder_output =  encoder_model.predict(input_lang_sequence)
  decoder_initial_states_value = encoder_output[1:]
  encoded_seqs = encoder_output[0]
  # Build the starting sequence for decoder model that is always a <start> as we defined this on each hindi sentences
  target_text_seq = np.zeros((1,1))
  target_text_seq[0][0] = decoder_tk.word_index["<start>"]

  ## Lets initialize the predicted sentence
  predicted_sentence = ""
  num_of_predictions = 0
  ## flag to check if prediction should be stop
  stop_loop = False
  ## Start the loop
  while not stop_loop:
    ## input value for decoder is H and C which we got from Encoder Predicted Model and Target Language start sequence which is always <start>
    predicted_outputs,h,c,a_matrix = decoder_model.predict([target_text_seq]+decoder_initial_states_value+[encoded_seqs])
    ## Now get the predicted word index with highest probability
    predicted_output = np.argmax(predicted_outputs[0,-1,:]) ## ignore index 0 and -1
    ## Now get the predicted word from predicted index
    predicted_word = int_to_word_decoder[predicted_output]
    ## Check if predcition should stop
    if(predicted_word == "<end>" or num_of_predictions>max_decoder_seq_length):
      stop_loop=True
      continue
    num_of_predictions += 1

    ## Updated predicted_sentences 
    if (len(predicted_sentence)==0):
      predicted_sentence = predicted_word
    else:
      predicted_sentence = predicted_sentence+" "+predicted_word
    ## Update target_lang_seq as input for decoder for next initilization for new predicted word
    target_text_seq[0][0] = predicted_output
    ## Need to update initial input value for decoder 
    decoder_initial_states_value = [h,c]
  return predicted_sentence

***Call the Prediction function using any random sentence***

In [69]:
#Generate a random number
start_num = np.random.randint(0, high=len(encoder_text) - 10)

#Predict model output for 5 sentences
for i in range(start_num, start_num + 5):
    input_sequence = encoder_input_data[i : i+1]
    predicted_sentence = decoder_output_sentence(input_sequence)
    print('--------')
    print ('Input sentence: ', encoder_text[i])
    print ('Predicted sentence: ', predicted_sentence )

--------
Input sentence:  I can't really talk about it.
Predicted sentence:  मैं उस बारे में कुछ बोल नहीं सकता।
--------
Input sentence:  I didn't want this to happen.
Predicted sentence:  मैं नहीं चाहता था कि ऐसा हो।
--------
Input sentence:  I don't feel like eating now.
Predicted sentence:  मुझे अभी खाना खाने का मन नहीं है।
--------
Input sentence:  I don't like to drink coffee.
Predicted sentence:  मुझे कॉफ़ी पीना पसंद नहीं।
--------
Input sentence:  I followed the deer's tracks.
Predicted sentence:  मैंने हिरण के पैरों के निशानों का पीछा किया।


***Save encoder and Decoder Model***

In [70]:
#Compile models to avoid error
encoder_model.compile(optimizer='adam',loss='categorical_crossentropy')
decoder_model.compile(optimizer='adam',loss='categorical_crossentropy')

#Save the models
encoder_model.save('model/seq2seq_encoder_eng_hin.hd5')  #Encoder model
decoder_model.save('model/seq2seq_decoder_eng_hin.hd5')  #Decoder model

INFO:tensorflow:Assets written to: model/seq2seq_encoder_eng_hin.hd5/assets
INFO:tensorflow:Assets written to: model/seq2seq_decoder_eng_hin.hd5/assets


***Save Encoder and Decoder Tokenizer***

In [0]:
## Need to save becuase we cannot change the indexing once we traiined our model.
import pickle

pickle.dump(encoder_tk,open('model/encoder_tokenizer_eng','wb'))
pickle.dump(decoder_tk,open('model/decoder_tokenizer_hin','wb'))